# Imports

In [26]:
import tensorflow as tf

# Dataset
import numpy as np
import pandas as pd
import json

# Constants

In [27]:
# Model
LEARNING_RATE = 5e-2
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS = tf.keras.losses.Huber()
METRICS = ["mae", "mse"]
EPOCHS = 1000

# Data split
TEST_SPLIT = 0.1                # Float or int
VALID_SPLIT = 0.1               # Float
TRAIN_SPLIT = 1 - VALID_SPLIT   # Float

# Dataset window
STEPS_SIZE = 15               # minutes
PREDICTS_SIZE = 1             # minute(s)
WINDOW_SIZE = STEPS_SIZE + PREDICTS_SIZE
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 64

# Dataset frame
NUM_OF_FEATURES = 6
NUM_OF_LABELS = 2

# File
DATASET_FILE_1 = './dataset/user_1/usr_1_w1_y2022.csv'
DATASET_FILE_2 = './dataset/user_1/usr_1_w2_y2022.csv'
model_1_file = './model/user_1/'
INFERENCE_DATASET_FILE = './dataset/user_1/a.json'






 
with open (INFERENCE_DATASET_FILE) as f:
  data = json.load(f)
INFERENCE_DATASET_FILE2 = json.dumps(data)

# Utility Functions

In [28]:
# Measure distance
def distance(coordinate_1:tuple[float,float], coordinate_2:tuple[float, float]) -> float:
    """Measure haversine distance between two coordinate.
    
    Note:
    - coordinate : tuple of latitude and longitude, ex. (3.10326051, 91.23206407)
    Reference: https://en.wikipedia.org/wiki/Haversine_formula
    """
    # constants
    earth_radius = 6371000 # in meters
    
    # unpack and convert params to radian
    lat_1, long_1 = np.radians(coordinate_1)
    lat_2, long_2 = np.radians(coordinate_2)
    
    d_lat = lat_2 - lat_1
    d_long = long_2 - long_1
    
    # calculate and return distance
    return 2 * earth_radius * np.arcsin (np.sqrt(
        np.sin(d_lat/2) ** 2
        + np.cos(lat_1) * np.cos(lat_2) * np.sin(d_long/2) ** 2))
    
distance((-6.200000, 106.816666),(-6.914744, 107.609810))
    

118291.96053744997

# Data

## Fetching

In [29]:
def fetch_dataset(source):
    if  source[-1] == 'v':
        data = pd.read_csv(source)
    else :
        meta = json.loads(source)
        data = pd.DataFrame(meta)
    return data

data = fetch_dataset(DATASET_FILE_1)
data

,date,time,latitude,longitude
0,3/7/2022,0:00:00,-6.268917,106.780112
1,3/7/2022,0:02:00,-6.268917,106.780112
2,3/7/2022,0:04:00,-6.268917,106.780112
3,3/7/2022,0:06:00,-6.268917,106.780112
4,3/7/2022,0:08:00,-6.268917,106.780112
...,...,...,...,...
5035,3/13/2022,23:50:00,-6.268917,106.779552
5036,3/13/2022,23:52:00,-6.268917,106.779552
5037,3/13/2022,23:54:00,-6.268917,106.779552
5038,3/13/2022,23:56:00,-6.268917,106.779552


In [16]:
f = open(INFERENCE_DATASET_FILE)
x = fetch_dataset(f)

TypeError: '_io.TextIOWrapper' object is not subscriptable

## Preprocessing

In [5]:
def preprocess_dataset(data):
    """Preprocess dataset by doing:
    1. convert date to multiple column
    2. convert time to cumulative minute
    3. rearrange fields"""
    # Converting date string to datetime
    data["date"] = pd.to_datetime(data["date"])
    data["day_of_week"] = data["date"].dt.day_of_week
    data["month"] = data["date"].dt.month
    data["year"] = data["date"].dt.year

    # Converting time to cumulative minute
    # source: https://stackoverflow.com/questions/17951820/convert-hhmmss-to-minutes-using-python-pandas
    # credit: Andy Hayden
    data["time"] = data["time"].str.split(':').apply(lambda time: int(time[0]) * 60 + int(time[1]))

    # Removing unused column
    del data["date"]

    # Rearrange column
    data = data[["year", "month", "day_of_week", "time", "latitude", "longitude"]]
    return data

data = preprocess_dataset(data)
data

,year,month,day_of_week,time,latitude,longitude
0,2022,3,0,0,-6.268917,106.780112
1,2022,3,0,2,-6.268917,106.780112
2,2022,3,0,4,-6.268917,106.780112
3,2022,3,0,6,-6.268917,106.780112
4,2022,3,0,8,-6.268917,106.780112
...,...,...,...,...,...,...
5035,2022,3,6,1430,-6.268917,106.779552
5036,2022,3,6,1432,-6.268917,106.779552
5037,2022,3,6,1434,-6.268917,106.779552
5038,2022,3,6,1436,-6.268917,106.779552


## Splitting

In [6]:
def split_dataset(data, train_split, test_split):
    """Split data to train, valid, and test data"""
    # Split train_valid data and test data
    test_len = test_split
    if type(test_split)==float:
        test_len = int(test_len * len(data))
    train_val_data, test_data = data[:-test_len], data[-test_len:]
    
    # Split train data and valid data
    train_len = int(len(train_val_data) * train_split)
    train_data, valid_data = train_val_data[:train_len], train_val_data[train_len:]
    
    return train_data, valid_data, test_data

train_data, valid_data, test_data = split_dataset(data, TRAIN_SPLIT, TEST_SPLIT)

print("Dataset Shape")
print(f'Train : {train_data.shape}')
print(f'Valid : {valid_data.shape}')
print(f'Test  : {test_data.shape}')

Dataset Shape
Train : (4082, 6)
Valid : (454, 6)
Test  : (504, 6)


## Windowing

In [7]:
def windowed_dataset(data, steps_size, predicts_size, batch_size, shuffle_buffer):
    """Create windowed dataset"""
    # Converting to tfds
    wds = tf.data.Dataset.from_tensor_slices(data)
    
    # Data shifting
    wds = wds.window(steps_size+predicts_size, shift=predicts_size, drop_remainder=True)
    
    # Flatten windows
    wds = wds.flat_map(lambda window : window.batch(steps_size+predicts_size))
    
    # Create window tuples
    wds = wds.map(lambda window: (window[:-predicts_size], window[-predicts_size:, -NUM_OF_LABELS:]))
    
    # Shuffle windows
    wds = wds.shuffle(shuffle_buffer)
    
    # Batch windows
    wds = wds.batch(batch_size).prefetch(1)
    
    return wds

wds = windowed_dataset(data, STEPS_SIZE, PREDICTS_SIZE, BATCH_SIZE, SHUFFLE_BUFFER_SIZE)
for idx,(x,y) in enumerate(wds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    break

x =  (32, 15, 6)
y =  (32, 1, 2)


## Building

In [8]:
def build_dataset(source, **kwargs):
    """Build dataset to make a train-ready dataset
    list of valid kwargs:
    - test_split: float - split value for test from whole dataset
    - valid_split: float - split value for valid from train_valid dataset
    - steps_size: int - number of steps used for prediction
    - predicts_size: int - number of predictions
    - batch_size: int - dataset batch size
    - shuffle_buffer_size: int - shuffle buffer size
    - num_of_features: int - number of features
    - num_of_labels: int - number of labels
    """
    # BUILD CONSTANTS
    # Data split
    test_split = kwargs.get('test_split', TEST_SPLIT)
    valid_split = kwargs.get('valid_split', VALID_SPLIT)
    train_split = 1 - valid_split

    # Dataset window
    steps_size = kwargs.get('steps_size', STEPS_SIZE)
    predicts_size = kwargs.get('predicts_size', PREDICTS_SIZE)
    window_size = steps_size + predicts_size
    batch_size = kwargs.get('batch_size', BATCH_SIZE)
    shuffle_buffer_size =  kwargs.get('shuffle_buffer_size', SHUFFLE_BUFFER_SIZE)

    # Dataset frame
    num_of_features = kwargs.get('num_of_features', NUM_OF_FEATURES)
    num_of_labels = kwargs.get('num_of_labels',NUM_OF_LABELS)
    
    # FETCHING DATASET
    ds = fetch_dataset(source) # use await for later asynchrounous usage
    
    # PREPROCESSING DATASET
    ds = preprocess_dataset(ds)
    
    # SPLITTING DATASET
    _train_ds, _valid_ds, _test_ds = split_dataset(ds, train_split, test_split)
    
    # WINDOWING AND RETURNING DATASET
    return \
        windowed_dataset(_train_ds, steps_size, predicts_size, batch_size, shuffle_buffer_size), \
        windowed_dataset(_valid_ds, steps_size, predicts_size, batch_size, shuffle_buffer_size), \
        windowed_dataset(_test_ds, steps_size, predicts_size, batch_size, shuffle_buffer_size)

train_wds, valid_wds, test_wds = build_dataset(DATASET_FILE_1)

# Model

## Base Model

In [40]:
def create_model():
    """Create Forecasting Model
    Model used: LSTM
    output should consist of 2 item, latitude and longitude
    """
    tf.keras.backend.clear_session()
    # Generating model
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, activation='sigmoid', input_shape=(STEPS_SIZE, NUM_OF_FEATURES), return_sequences=True),
        tf.keras.layers.LSTM(32, activation='sigmoid', return_sequences=True),
        tf.keras.layers.LSTM(16, activation='sigmoid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation='sigmoid'),
        tf.keras.layers.Dense(8, activation='sigmoid'),
        tf.keras.layers.Dense(NUM_OF_LABELS, activation='linear')
    ])

    # Compiling model
    model.compile(
        loss=LOSS,
        optimizer=OPTIMIZER,
        metrics=METRICS,
    )
    
    return model
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 64)            18176     
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            12416     
                                                                 
 lstm_2 (LSTM)               (None, 16)                3136      
                                                                 
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 2)                 1

## Training

In [41]:
model.fit(train_wds, epochs=20, validation_data=valid_wds)

Epoch 1/20
128/128 [==============================] - 6s 23ms/step - loss: 47.0927 - mae: 47.4660 - mse: 4439.8306 - val_loss: 39.7400 - val_mae: 40.0193 - val_mse: 3198.2400
Epoch 2/20
128/128 [==============================] - 2s 18ms/step - loss: 33.1852 - mae: 33.4589 - mse: 2265.1064 - val_loss: 26.4950 - val_mae: 26.7840 - val_mse: 1430.4080
Epoch 3/20
128/128 [==============================] - 2s 17ms/step - loss: 20.0276 - mae: 20.3044 - mse: 851.7085 - val_loss: 12.2990 - val_mae: 12.5829 - val_mse: 314.8916
Epoch 4/20
128/128 [==============================] - 2s 17ms/step - loss: 3.3625 - mae: 3.5641 - mse: 57.4533 - val_loss: 0.0010 - val_mae: 0.0388 - val_mse: 0.0020
Epoch 5/20
128/128 [==============================] - 2s 18ms/step - loss: 4.0684e-04 - mae: 0.0208 - mse: 8.1081e-04 - val_loss: 0.0012 - val_mae: 0.0367 - val_mse: 0.0024
Epoch 6/20
128/128 [==============================] - 2s 18ms/step - loss: 3.4069e-04 - mae: 0.0192 - mse: 6.8032e-04 - val_loss: 0.0014 -

## Evaluating Model

In [42]:
model.evaluate(test_wds)

16/16 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0513 - mse: 0.0045


[0.0022662500850856304, 0.051327671855688095, 0.004546389915049076]

## Using Model

### Converting Data

In [43]:
def convert_data(data):
    """Convert data to model input"""
    # Take last "steps size" data from copy data
    cdata = data.copy()[-STEPS_SIZE:]
    if len(cdata) != STEPS_SIZE:
        # Not enough data to do prediction
        return None
    # Add empty row in the bottom
    cdata.loc[cdata.shape[0]] = np.zeros(NUM_OF_FEATURES)
    
    return windowed_dataset(cdata, STEPS_SIZE, PREDICTS_SIZE, BATCH_SIZE, SHUFFLE_BUFFER_SIZE)

predict_data = build_inference_dataset(INFERENCE_DATASET_FILE)
predict_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>

### Build Inference Input (Dataset)

In [30]:
def build_inference_dataset(source, **kwargs):
    """Build inference dataset in the form of prefetch dataset from given source"""
    # BUILD CONSTANTS
    # Data split
    test_split = kwargs.get('test_split', TEST_SPLIT)
    valid_split = kwargs.get('valid_split', VALID_SPLIT)
    train_split = 1 - valid_split

    # Dataset window
    steps_size = kwargs.get('steps_size', STEPS_SIZE)
    predicts_size = kwargs.get('predicts_size', PREDICTS_SIZE)
    window_size = steps_size + predicts_size
    batch_size = kwargs.get('batch_size', BATCH_SIZE)
    shuffle_buffer_size =  kwargs.get('shuffle_buffer_size', SHUFFLE_BUFFER_SIZE)

    # Dataset frame
    num_of_features = kwargs.get('num_of_features', NUM_OF_FEATURES)
    num_of_labels = kwargs.get('num_of_labels',NUM_OF_LABELS)
    
    # FETCHING DATASET
    ds = fetch_dataset(source) # use await for later asynchrounous usage
    ds = ds[-steps_size:] # take only n-steps
    
    # PREPROCESSING DATASET
    ds = preprocess_dataset(ds)
    
    # CONVERTING TO INFERENCE SHAPE
    if len(ds) != steps_size:
        return None
    # Add empty row as empty label
    ds.loc[ds.shape[0]] = np.zeros(num_of_features)
    
    # WINDOWING AND RETURNING DATASET
    return windowed_dataset(ds, steps_size, predicts_size, batch_size, shuffle_buffer_size)

predict_data = build_inference_dataset(INFERENCE_DATASET_FILE2)
predict_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>

### Predicting Data

In [45]:
def predict(model, data):
    """Predict data using model and data"""
    steps_data = convert_data(data)
    if not steps_data:
        # If converting failed (not enough data) return none
        return None
    # Else return prediction
    return  model.predict(
        steps_data
    )[0]


print("\n\n========================================")
print("Predict")
print("========================================\n")
print(predict(model, data[:5000]))

print("\n\n========================================")
print("Label")
print("========================================\n")
print([data.loc[5000]["latitude"], data.loc[5000]["longitude"]])



Predict

1/1 [==============================] - 0s 374ms/step
[ -6.1883955 106.77954  ]


Label

[-6.268917097, 106.7795517]


## Trigger

In [46]:
# Constants
max_range = 1000 # in meters

# Trigger function
def calculate_trigger(
        coordinate_1:tuple[float,float],
        coordinate_2:tuple[float,float],
        max_range:float) -> bool:
    """Calculate if distance between two coordinates is 
    over the max range and return True if distance is more
    than max_range"""
    print(distance(coordinate_1, coordinate_2))
    return distance(coordinate_1, coordinate_2) > max_range

calculate_trigger(
    predict(model, data[:5000]),
    (data.loc[5000]["latitude"], data.loc[5000]["longitude"]),
    max_range)

1/1 [==============================] - 0s 44ms/step
8953.585736508117


True

# References

- [Sequences, Time Series and Prediction by DeepLearning.AI](https://www.coursera.org/learn/tensorflow-sequences-time-series-and-prediction)
- [Multi-Variate Time Series Forecasting Tensorflow by Nicholas Jhana](https://www.kaggle.com/code/nicholasjhana/multi-variate-time-series-forecasting-tensorflow/notebook#Visualizing-Predictions)